In [5]:
data=open('flare').read()
# 去空格
data= data.replace('\n','').replace('\r','')
#字符去重
letters=list(set(data))
num_letters=len(letters)
print(letters)

['H', 'S', ' ', 'u', 'l', 'e', 'i', 'y', 'n', 'r', 'h', 'f', 'a', 'o', 'd', 't', 's', 'm', 'A', 'b', 'c', '.', 'p']


In [6]:
#建立字典
int_to_char={a:b for a,b in enumerate(letters)}
print(int_to_char)

{0: 'H', 1: 'S', 2: ' ', 3: 'u', 4: 'l', 5: 'e', 6: 'i', 7: 'y', 8: 'n', 9: 'r', 10: 'h', 11: 'f', 12: 'a', 13: 'o', 14: 'd', 15: 't', 16: 's', 17: 'm', 18: 'A', 19: 'b', 20: 'c', 21: '.', 22: 'p'}


In [7]:
char_to_int={b:a for a,b in enumerate(letters)}
print(char_to_int)

{'H': 0, 'S': 1, ' ': 2, 'u': 3, 'l': 4, 'e': 5, 'i': 6, 'y': 7, 'n': 8, 'r': 9, 'h': 10, 'f': 11, 'a': 12, 'o': 13, 'd': 14, 't': 15, 's': 16, 'm': 17, 'A': 18, 'b': 19, 'c': 20, '.': 21, 'p': 22}


In [8]:
time_step=20

In [16]:
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data,slide):
    x=[]
    y=[]
    for i in range(len(data)-slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y,char_to_int):
    x_to_int=[]
    y_to_int=[]
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])
    return x_to_int,y_to_int
#实现输入字符文章的批量处理，输入整个字符，滑动窗口大小，转化字典
def data_preprocessing(data,slide,num_letters,char_to_int):
    char_Data=extract_data(data,slide)
    int_Data=char_to_int_Data(char_Data[0],char_Data[1],char_to_int)
    Input=int_Data[0]
    Output=list(np.array(int_Data[1]).flatten())
    Input_RESHAPED=np.array(Input).reshape(len(Input),slide)
    new=np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:]=to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new,Output

In [17]:
 X,y=data_preprocessing(data,time_step,num_letters,char_to_int)

In [19]:
X.shape

(56148, 20, 23)

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=10)

In [21]:
X_train.shape

(50533, 20, 23)

In [22]:
y_train_category=to_categorical(y_train,num_letters)
print(y_train_category)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [24]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
model=Sequential()
model.add(LSTM(units=20,input_shape=(X_train.shape[1],X_train.shape[2]),activation='relu'))
model.add(Dense(units=num_letters,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 23)                483       
Total params: 4,003
Trainable params: 4,003
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X_train,y_train_category,batch_size=1000,epochs=5)

Epoch 1/5
50533/50533 [==============================] - 6s 117us/step - loss: 0.7067 - accuracy: 0.8063
Epoch 2/5
50533/50533 [==============================] - 6s 126us/step - loss: 0.4010 - accuracy: 0.8919
Epoch 3/5
50533/50533 [==============================] - 6s 126us/step - loss: 0.1994 - accuracy: 0.9558
Epoch 4/5
50533/50533 [==============================] - 7s 132us/step - loss: 0.1283 - accuracy: 0.9823
Epoch 5/5
50533/50533 [==============================] - 7s 131us/step - loss: 0.1182 - accuracy: 0.9842


In [26]:
y_train_predict=model.predict_classes(X_train)
print(y_train_predict)

[ 2  5 20 ...  4  5  2]


In [29]:
y_train_predict_char=[int_to_char[i] for i in y_train_predict]

In [30]:
y_train_predict_char

[' ',
 'e',
 'c',
 'h',
 's',
 'a',
 'a',
 ' ',
 ' ',
 'r',
 'e',
 't',
 'u',
 'n',
 ' ',
 'i',
 'i',
 ' ',
 '.',
 ' ',
 't',
 ' ',
 'y',
 'a',
 't',
 't',
 '.',
 ' ',
 ' ',
 't',
 'u',
 'i',
 'c',
 ' ',
 ' ',
 'h',
 ' ',
 's',
 ' ',
 's',
 'h',
 't',
 'e',
 '.',
 ' ',
 's',
 'n',
 'h',
 'r',
 'r',
 '.',
 's',
 'u',
 ' ',
 ' ',
 'h',
 'a',
 'r',
 'd',
 'i',
 'i',
 ' ',
 'l',
 's',
 'l',
 'e',
 'i',
 'a',
 'n',
 'h',
 'r',
 ' ',
 'a',
 'a',
 'u',
 '.',
 'i',
 'e',
 'i',
 'h',
 'u',
 ' ',
 'a',
 'a',
 'r',
 ' ',
 'i',
 ' ',
 'u',
 'A',
 'A',
 'n',
 'h',
 'p',
 't',
 ' ',
 't',
 ' ',
 'u',
 ' ',
 'o',
 'i',
 ' ',
 ' ',
 ' ',
 's',
 'c',
 't',
 ' ',
 'y',
 'a',
 's',
 'A',
 'r',
 'u',
 ' ',
 'e',
 's',
 ' ',
 'h',
 'r',
 'h',
 'n',
 'e',
 's',
 'e',
 'i',
 'r',
 'a',
 ' ',
 ' ',
 'a',
 'i',
 't',
 'i',
 't',
 ' ',
 'n',
 ' ',
 'c',
 'l',
 'c',
 'i',
 'i',
 ' ',
 'n',
 ' ',
 'r',
 's',
 's',
 ' ',
 'c',
 'e',
 'l',
 ' ',
 'a',
 'u',
 '.',
 'i',
 ' ',
 ' ',
 'i',
 't',
 ' ',
 'i',
 'n',
 'n'

In [32]:
from sklearn.metrics import accuracy_score
accuracy_train=accuracy_score(y_train,y_train_predict)
accuracy_train

0.8063839471236618

In [33]:
y_test_predict=model.predict_classes(X_test)
accuracy_test=accuracy_score(y_test,y_test_predict)
accuracy_test

0.8096170970614426

In [34]:
y_test_predict_char=[int_to_char[i] for i in y_test_predict]
y_test_predict_char

[' ',
 'h',
 ' ',
 'a',
 'u',
 ' ',
 ' ',
 'l',
 'n',
 'd',
 'A',
 'r',
 'i',
 'y',
 'i',
 'n',
 'e',
 't',
 'i',
 'h',
 ' ',
 'a',
 'u',
 'h',
 'i',
 ' ',
 'e',
 'p',
 'n',
 'u',
 's',
 'i',
 'i',
 'u',
 't',
 'a',
 'c',
 ' ',
 'u',
 'i',
 ' ',
 's',
 'y',
 'u',
 'A',
 'u',
 'r',
 ' ',
 'A',
 'i',
 'o',
 ' ',
 ' ',
 'a',
 't',
 ' ',
 ' ',
 'i',
 'a',
 's',
 't',
 '.',
 'u',
 ' ',
 'i',
 ' ',
 'i',
 'i',
 'i',
 ' ',
 'i',
 'i',
 'A',
 ' ',
 ' ',
 'a',
 'o',
 'i',
 'u',
 'e',
 'e',
 'h',
 'l',
 ' ',
 'i',
 'e',
 'i',
 't',
 'e',
 'n',
 'a',
 'A',
 'i',
 'h',
 'a',
 'h',
 'r',
 ' ',
 't',
 's',
 ' ',
 'i',
 ' ',
 's',
 'e',
 'n',
 'a',
 ' ',
 '.',
 ' ',
 ' ',
 'l',
 ' ',
 'h',
 ' ',
 't',
 'i',
 'c',
 'h',
 't',
 ' ',
 ' ',
 'e',
 'h',
 's',
 's',
 'r',
 'A',
 'a',
 'i',
 'n',
 't',
 'a',
 's',
 'i',
 'n',
 'd',
 'e',
 'n',
 'e',
 'i',
 'a',
 'e',
 '.',
 'u',
 'a',
 ' ',
 'c',
 's',
 'n',
 'r',
 'i',
 't',
 'i',
 'i',
 '.',
 'a',
 's',
 'i',
 'n',
 'h',
 'h',
 ' ',
 ' ',
 'u',
 'a',
 'i'

In [35]:
new_letters='flare is a teacher in ai industry. He obtained his phd in Australia.'

In [41]:
X_new,y_new=data_preprocessing(new_letters,time_step,num_letters,char_to_int)
y_new_predict=model.predict_classes(X_new)

In [42]:
y_new_predict_char=[int_to_char[i] for i in y_new_predict]
y_new_predict_char

['n',
 ' ',
 'a',
 'i',
 ' ',
 'i',
 'n',
 'd',
 'u',
 's',
 't',
 'r',
 'y',
 '.',
 ' ',
 'H',
 'e',
 ' ',
 'o',
 'b',
 't',
 'a',
 'i',
 'n',
 'e',
 'd',
 ' ',
 'h',
 'i',
 's',
 ' ',
 'p',
 'h',
 'd',
 ' ',
 'i',
 'n',
 ' ',
 'A',
 'u',
 's',
 't',
 'r',
 'a',
 'l',
 'i',
 'a',
 '.']

In [44]:
for i in range(0,X_new.shape[0]-20):
    print(new_letters[i:i+20],'--predict next letter is --',y_new_predict_char[i])

flare is a teacher i --predict next letter is -- n
lare is a teacher in --predict next letter is --  
are is a teacher in  --predict next letter is -- a
re is a teacher in a --predict next letter is -- i
e is a teacher in ai --predict next letter is --  
 is a teacher in ai  --predict next letter is -- i
is a teacher in ai i --predict next letter is -- n
s a teacher in ai in --predict next letter is -- d
 a teacher in ai ind --predict next letter is -- u
a teacher in ai indu --predict next letter is -- s
 teacher in ai indus --predict next letter is -- t
teacher in ai indust --predict next letter is -- r
eacher in ai industr --predict next letter is -- y
acher in ai industry --predict next letter is -- .
cher in ai industry. --predict next letter is --  
her in ai industry.  --predict next letter is -- H
er in ai industry. H --predict next letter is -- e
r in ai industry. He --predict next letter is --  
 in ai industry. He  --predict next letter is -- o
in ai industry. He o --predict 